In [1]:
import redcap
import csv
import datetime
import pandas as pd
import pandasql as psql
from datetime import datetime, timedelta


# REDCap API endpoint and token
data = {
    'token': 'DE12CA04415CF93BCD68321EBF5040FD',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
df= redcap.Project('https://redcap.mrc.gm:8443/redcap/api/',data['token'])

data = {
    'token': 'B5C8DD01C47A66E81E154A8AC735D281',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
ck_wk=redcap.Project('https://redcap.mrc.gm:8443/redcap/api/',data['token'])

data = {
    'token': 'D6807C091B1FC14F57CC9B2972D0B398',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
indigo=redcap.Project('https://ruff.mrc.gm:8443/redcap/api/',data['token'])




In [2]:
sen_data=ck_wk.export_report(report_id='10068')
sen_log_data=pd.DataFrame(sen_data)
sen_log_data=sen_log_data[['wk_ckno','enu_name','enu_village', 'enu_comp','sen_contact1','sen_contact2','sen_contact3']]
sen_log_data=sen_log_data.rename(columns={'wk_ckno': 'ck_wkno'})
village_mapping = {
'001':	'DUMBUTO',
'002':	'SANKANDI',
'003':	'NIORO JATTABA',
'004':	'JATTABA',
'005':	'JIFFARONG',
'006':	'BAJANA',
'007':	'KULI KUNDA',
'008':	'JAMARU',
'009':	'BRIKAMANDING',
'010':	'KANTONG KUNDA',
'011':	'JALI',
'013':	'MANDUAR',
'014':	'BANG KULING',
'015':	'GISSAY',
'016':	'TANKULAR',
'017':	'JOLI',
'018':	'KUYANG',
'019':	'BANTASU',
'020':	'SANTAMBA',
'021':	'MISSIRA',
'022':	'TABORANGKOTO',
'023':	'BURONG',
'024':	'JULA KUNDA',
'025':	'KARANTABA',
'026':	'MANDINA',
'027':	'JANNEH KUNDA',
'028':	'KEMOTO',
'029':	'KENEBA',
'030':	'BATELLING',
'031':	'SANDENG',
'032':	'WUDEBA',
'034':	'KENOKOTO',
'035':	'MANARI',
'036':	'NINETEEN',
'040':	'WUROKANG',
'041':	'KWINELLA SANSANKONO',
'042':	'KWINELLA NIA KUNDA',
'043':	'TENDABA',
'044':	'BUMARR',
'045':	'BAMBAKO',
'046':	'KUNDONG MARIAYA',
'047':	'NEMA',
'048':	'KUNDANG NUMU KUNDA',
'049':	'KUNDANG FULA KUNDA',
'050':	'NEMA KUTA',
'051':	'JIRROFF',
'052':	'MADINA ANGALLEH',
'053':	'JATTA KUNDA',
'054':	'MANDINA CENTRAL',
'055':	'SARE SARJO',
'056':	'SIBETO',
'057':	'SARE NDALLA',
'058':	'TABANANI',
'060':	'WILLINGARA',
'061':	'SARE MAMUDU',
'070':	'KOLIOR',
'071':	'JOMARR',
'072':	'JASOBO',
'073':	'SARE MUSA',
'074':	'YORRO JULA',
'075':	'BABOU YAA',
'076':	'MASEMBEH',
'077':	'GENIERE',
'078':	'KAIAF',
'079':	'MADINA KAIAF',
'080':	'SARE SAMBA',
'081':	'MADINA CEESAY KUNDA',
'082':	'NJOLFEN',
'083':	'TORANKA BANTANG',
'084':	'SAREH PATEH',
'085':	'MUNKUTALA'
}

# Assuming sen_log_data is your DataFrame
sen_log_data['enu_village'] = sen_log_data['enu_village'].map(village_mapping)

In [3]:
identifier=df.export_records(forms=['consent'])
identifier=pd.DataFrame(identifier)
identifier=identifier[['participant_id','ck_wkno','consent_complete']]
identifier=identifier[
    (identifier['consent_complete']=='2')
]

In [4]:
#export all project data
df_data=df.export_records(forms=['withdrawal'])
df_data=pd.DataFrame(df_data)
# Specify the fields you want to extract base on logics
df_withdrw=df_data[['participant_id','wd_vdadte_q1','wd_reason_q5___2','redcap_event_name','withdrawal_complete']]
df_withdrw=pd.DataFrame(df_withdrw)
df_withdrw = df_withdrw[
    (df_withdrw['withdrawal_complete']=='2')]

In [5]:
df_delivery=df.export_records(forms=['delivery'])
df_delivery=pd.DataFrame(df_delivery)

In [6]:

df_delivery=df_delivery[['participant_id','del_sex_q10','del_vdate_q1','delivery_complete']]
df_delivery['del_vdate_q1'] = pd.to_datetime(df_delivery['del_vdate_q1'])
df_delivery=df_delivery[
    (df_delivery['delivery_complete']=='1')|
    (df_delivery['delivery_complete']=='2')

]
df_delivery = df_delivery[~df_delivery['participant_id'].isin(df_withdrw['participant_id'])]


In [7]:
df_infant_anthropometry=df.export_records(forms=['infant_anthropometry'])
df_infant_anthropometry=pd.DataFrame(df_infant_anthropometry)
df_infant_anthropometry=df_infant_anthropometry[['participant_id','ia_q1','ia_month','infant_anthropometry_complete']]
df_infant_anthropometry['ia_q1'] = pd.to_datetime(df_infant_anthropometry['ia_q1'])
df_infant_anthropometry['ia_month'] = pd.to_numeric(df_infant_anthropometry['ia_month'])
df_infant_anthropometry=df_infant_anthropometry[
     (df_infant_anthropometry['infant_anthropometry_complete']=='1')|
    (df_infant_anthropometry['infant_anthropometry_complete']=='2')
]

In [8]:

# Assuming 'del_vdate_q1' and 'participant_id' are columns in your DataFrame 'df_delivery'
base_dates = pd.to_datetime(df_delivery['del_vdate_q1'])
# Create an empty DataFrame to store the results
result_df = pd.DataFrame()

# Create lists to store calculated values
base_date_list = []
due_date_list = []
last_due_date_list = []
participant_id_list = []
month_list = []
# Calculate due dates for the next 12 months for each base date
for i, base_date in enumerate(base_dates):
    for month_num in range(1, 13):
        base_date_list.append(base_date)
        due_date = base_date + timedelta(days=(30 + (month_num - 1) * 30))
        due_date_list.append(due_date)
        last_due_date = due_date + timedelta(days=7)
        last_due_date_list.append(last_due_date)
        participant_id_list.append(df_delivery['participant_id'].iloc[i])
        month_list.append(month_num)
     

# Add the lists to the result DataFrame
result_df['participant_id'] = participant_id_list
result_df['delivery_date'] = base_date_list
result_df['due_date'] = due_date_list
result_df['last_due_date'] = last_due_date_list
result_df['ia_month'] = month_list

In [9]:
from datetime import datetime, timedelta
import pandas as pd

# Get the current date
current_date = datetime.now()

# Assuming 'result_df' is the DataFrame with due dates and 'df_infant_anthropometry' contains 'ia_month' information
# Merge the two DataFrames on 'participant_id' and 'ia_month' columns
merged_df = result_df.merge(df_infant_anthropometry, on=['participant_id', 'ia_month'], how='left', indicator=True)

# Filter the rows where 'ia_month' exists in df_infant_anthropometry (indicating completion)
incomplete_df = merged_df[merged_df['_merge'] != 'both']

# Drop the '_merge' column
incomplete_df = incomplete_df.drop(columns=['_merge'])

# Add a new column based on the 'last_due_date' comparison
incomplete_df['overdue_status'] = incomplete_df.apply(
    lambda row: 'overdue for this month' if row['last_due_date'] <= current_date else '', axis=1
)

# Calculate the last day of the current month
last_day_of_month = (current_date.replace(day=1) + pd.DateOffset(months=1) - timedelta(days=1))

# Filter the 'incomplete_df' to keep only the rows with 'due_date' until the current month
incomplete_df = incomplete_df[incomplete_df['due_date'] <= last_day_of_month]

incomplete_df = incomplete_df[['participant_id', 'delivery_date', 'due_date', 'last_due_date', 'ia_month', 'overdue_status']]

Margh_1 = pd.merge(identifier, incomplete_df, on='participant_id', how='inner')
Margh_2 = pd.merge(Margh_1, sen_log_data, on='ck_wkno', how='inner')
Margh_2 = Margh_2[['participant_id', 'ck_wkno', 'enu_name', 'enu_village', 'delivery_date',
                   'due_date', 'last_due_date', 'ia_month', 'overdue_status',
                   'enu_comp', 'sen_contact1', 'sen_contact2']]


indigo_eden_participant=indigo.export_records(forms=['consent'])
indigo_eden_participant=pd.DataFrame(indigo_eden_participant)
indigo_eden_participant=indigo_eden_participant[['con_participant_eden_num_q11']]
indigo_eden_participant=indigo_eden_participant.rename(columns={'con_participant_eden_num_q11':'participant_id'})
indigo_eden_participant=indigo_eden_participant[
   (indigo_eden_participant['participant_id']!="")
]
Margh_2 = Margh_2[~Margh_2['participant_id'].isin(indigo_eden_participant['participant_id'])].sort_values(by='overdue_status', na_position='first')
Margh_2.to_csv('infant_antrop_callist.csv', index=False)


In [10]:

# Filter pregnancy test results
pregnancy_result = df.export_records(forms=['pregnancy_test_result'])
pregnancy_result = pd.DataFrame(pregnancy_result)
pregnancy_result = pregnancy_result[['participant_id',
       'redcap_repeat_instance', 'prg_cycle', 'pt_testdate_q1','pt_pregnancy_positive_q3', 'ptr_completedby_q4',
       'pregnancy_test_result_complete']]
pregnancy_result = pregnancy_result[
    (pregnancy_result['pt_pregnancy_positive_q3'] == '1')
]

# Filter ultrasound scan records
ultrasound_scan = df.export_records(forms=['ultrasound_scan'])
ultrasound_scan = pd.DataFrame(ultrasound_scan)
ultrasound_scan = ultrasound_scan[['participant_id', 'ultrascan_date', 'ultrascan_exp_date', 'conception_date', 'ultrasound_scan_complete']]
ultrasound_scan = ultrasound_scan[
    (ultrasound_scan['ultrasound_scan_complete'].isin(['1', '2']))
]

# Merge pregnancy results and ultrasound scans
merged_data = pd.merge(pregnancy_result, ultrasound_scan, on='participant_id', how='inner')

# Create a new column to store the ultrascan_exp_date or NaN for each record
merged_data['ultrascan_exp_date_value'] = merged_data['ultrascan_exp_date'].apply(lambda x: x if x.strip() else None)

# Drop rows with NaN in the 'ultrascan_exp_date_value' column
merged_data.dropna(subset=['ultrascan_exp_date_value'], inplace=True)

# Drop duplicates based on the 'participant_id' column
merged_data.drop_duplicates(subset=['participant_id'], keep='first', inplace=True)
merged_data.to_csv('HCG.csv',index=False)

In [11]:
CONSENT_report=df.export_report(report_id='10351')
CONSENT_report=pd.DataFrame(CONSENT_report)
CONSENT_report=CONSENT_report[
    (CONSENT_report['cconsent']=='1')
]

In [12]:
consent_form_1= CONSENT_report[~CONSENT_report['participant_id'].isin(df_withdrw['participant_id'])]

In [13]:
village_mapping = {
'1':'Amie Joof',
'1':'Amie Joof',
'2':'Lamin K Jawla',
'3':'Demba Jallow',
'4':'Foday Sanyang',
'5':'Lamin Minteh',
'6':'Modou Bah',
'7':'Amadou Jatta',
'8':'8. Sidu Sibi',
'9':'Kassa Kora',
'10':'Lamin Njai',
'11':'Lamin Jatta',
'12':'Fabakary Njie',
'13':'Mustapha Joof',
'14':'Omar Bah',
'15':'Modou Lamin Njie',
'16':'Alieu Bah',
'17':'Tumbulu Drammeh',
'18':'Buba Ceesay',
'19':'Foday k Darboe',
'20':'Ansumana Barjo',
'21':'Lamin L K Ceesay',
'22':'Amat Joof',
'23':'Yerro Sowe',
'24':'Ousman Kambi',
'25':'Kemo Y Ceesay'

    # Add more mappings as needed
}

# Assuming sen_log_data is your DataFrame
consent_form_1['consent_completedby_q8'] = consent_form_1['consent_completedby_q8'].map(village_mapping)

C:\Users\abliebah\AppData\Local\Temp\ipykernel_3756\755510290.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consent_form_1['consent_completedby_q8'] = consent_form_1['consent_completedby_q8'].map(village_mapping)


In [14]:
consent_form_1.to_csv('consent_form_1.csv',index=False)

In [15]:
consent=df.export_records(forms=['consent'])
consent=pd.DataFrame(consent)

In [16]:
consent=consent[['participant_id', 'redcap_event_name','redcap_repeat_instance', 'ck_wkno', 'cstaffid','consent_form_date_q10', 'consent_complete']]

In [17]:
consent=consent[
    (consent['redcap_event_name']=='recruitment_arm_1')&
    (consent['consent_complete']=='2')
]

In [18]:
consent.to_csv('consent.csv',index=False)